# Encoding Audio Using Least Significant Bit

In [4]:
#importing libraries
import numpy as np
import IPython.display as ipd
import soundfile
from bitstring import Bits

Used the code below to create an generated audio file that says 'Father Christmas doesn't exist.'.  I commented it out since there is no need to re-run it.

In [5]:
# from gtts import gTTS
# from pydub import AudioSegment
# from os import path

# mytext = 'Father Christmas does not exist.'

# language = 'en'

# myobj = gTTS(text=mytext, lang=language, slow=False)
# myobj.save("secret_3.mp3")
# sound = AudioSegment.from_mp3('./sounds/secret_3.mp3')
# sound.export("secret_3.wav", format="wav")

In [6]:
# opening the carrier and message files
carrier_5, carrier_sr_5 = soundfile.read('./sounds/Ex3_sound5.wav', dtype=np.int16)
message_5, message_sr_5 = soundfile.read('./sounds/secret_3.wav', dtype=np.int16)
message_5 = np.hstack((message_5, message_5, message_5, message_5, message_5))

Here is the original audio:

In [7]:
ipd.Audio('./sounds/Ex3_sound5.wav')

Here is the secret code message: 

In [8]:
ipd.Audio('./sounds/secret_3.wav')

Encoding Algorithm:

In [9]:
from bitstring import Bits

# inspired by : https://colab.research.google.com/drive/1pkeMzy6g29pHtFmG1yltpOavGVdY0B60
# changed the code and removed a part of the code that unnecessarily made sure the length of the string is divisible 

def encode_least_significant_bit(carrier, data, n_bits=1):
  # converting all integer values of secret message to binary strings:
  secret_code = []
  for value in np.nditer(data):
    secret_code.append(np.binary_repr(value, 16))

  # connecting the bits together to create a string
  secret_code = ''.join(secret_code)

  # making sure carrier size is equal to the secret code size
  secret_code = secret_code.ljust(carrier.size * n_bits, '0')[:carrier.size * n_bits]

  # Encoding the signal here by modifying the least significant bit
  audio_with_hidden_data = np.zeros(carrier.shape, dtype=carrier.dtype)
  for i in range(len(carrier)):
    # Convert ith value of carrier to binary string:
    binary_string = np.binary_repr(carrier[i], 16)
    # Set the last bit of the binary string to be a bit from the secret message:
    altered_binary = binary_string[:-n_bits] + secret_code[i*n_bits:i*n_bits+n_bits]
    audio_with_hidden_data[i] = Bits(bin=altered_binary).int # Binary string to int

  return audio_with_hidden_data

In [10]:
audio_with_code_5 = encode_least_significant_bit(carrier_5, message_5, 10)
soundfile.write('audio_with_hidden_message_5.wav', audio_with_code_5, carrier_sr_5)
ipd.Audio('audio_with_hidden_message_5.wav')

Decoding Algorithm:

In [11]:
# inspired by : https://colab.research.google.com/drive/1pkeMzy6g29pHtFmG1yltpOavGVdY0B60
# redefining decode least significant bit to decode using n_bit of 10 
def decode_least_significant_bits(signal, n_bits=1):

  # collecting the least significant bit here
  secret_bits = []
  for value in np.nditer(signal):
    ls_bit = np.binary_repr(value, 16)[-n_bits:]
    secret_bits.append(ls_bit)

   # connecting the bits together to create a string
  secret_bits = ''.join(secret_bits)
  
  # converting 16 bit chunks to 64 bits 
  retrieved_audio = np.zeros(len(secret_bits) // 16, dtype=np.int16)
  for i in range(retrieved_audio.size):
    retrieved_audio[i] = Bits(bin=secret_bits[i*16:(i+1)*16]).int

  return retrieved_audio

File successfully decoded:

In [12]:
retrieved_hidden_message_3 = decode_least_significant_bits(audio_with_code_5, 10)
soundfile.write('retrieved_hidden_message_5.wav', retrieved_hidden_message_3, message_sr_5)
ipd.Audio('retrieved_hidden_message_5.wav')